# 0.1 라이브러리 임포트

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable
import numpy as np

# 0.2  하이퍼파라미터 세팅

In [3]:
# Hyper-parameters
batch_size = 128
num_epochs = 10

word_vec_size = 256
dropout_p = 0.3

hidden_size = 512
num_layers = 4

## yhk 추가
learning_rate = 0.001 # 디폴트 0.001

In [5]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. SMS train, test dataset 가져오기

In [6]:
from data_loader import DataLoader

In [10]:
loaders = DataLoader(
    train_fn = './sms.maxlen.uniq.shuf.train.tsv',
    batch_size = batch_size,
    valid_ratio = .2,
    device = -1,
    max_vocab = 999999,
    min_freq = 5,
)

In [11]:
test_loader = DataLoader(
    train_fn = './sms.maxlen.uniq.shuf.train.tsv',
    batch_size = batch_size,
    valid_ratio = .01, #  모두 train
    device = -1,
    max_vocab = 999999,
    min_freq = 5,
)

# 2. 대략적인 데이터 형태

In [13]:
print("|train| =", len(loaders.train_loader.dataset),
     '|valid| =', len(loaders.valid_loader.dataset))

vocab_size = len(loaders.text.vocab)
num_classes = len(loaders.label.vocab)
print('|vocab| =', vocab_size, '|classes| =', num_classes)

('|train| =', 3722, '|valid| =', 930)
('|vocab| =', 1576, '|classes| =', 2)


# 3. 데이터 로드함수
   학습시킬 때 batch_size단위로 끊어서 로드하기 위함

# 데이터 로드함수 이해하기

In [16]:
n = 3 # 샘플로 출력할 데이터 개수
for i, data in enumerate(loaders.train_loader):
    labels = data.label
    texts = data.text
    
    if i > n:
        break
    print( "[%d]" %i)
    print("한 번에 로드되는 데이터 크기:", len(labels))
    
    # 출력
    for j in range(n):
        label = labels[j].numpy() # tensor -> numpy 로 변환
        text = texts[j].numpy()
        print( "label: ", label)
        print("text: ", text.shape)

[0]
('\xed\x95\x9c \xeb\xb2\x88\xec\x97\x90 \xeb\xa1\x9c\xeb\x93\x9c\xeb\x90\x98\xeb\x8a\x94 \xeb\x8d\xb0\xec\x9d\xb4\xed\x84\xb0 \xed\x81\xac\xea\xb8\xb0:', 128)
('label: ', array(0))
('text: ', (6,))
('label: ', array(0))
('text: ', (6,))
('label: ', array(0))
('text: ', (6,))
[1]
('\xed\x95\x9c \xeb\xb2\x88\xec\x97\x90 \xeb\xa1\x9c\xeb\x93\x9c\xeb\x90\x98\xeb\x8a\x94 \xeb\x8d\xb0\xec\x9d\xb4\xed\x84\xb0 \xed\x81\xac\xea\xb8\xb0:', 128)
('label: ', array(0))
('text: ', (9,))
('label: ', array(0))
('text: ', (9,))
('label: ', array(0))
('text: ', (9,))
[2]
('\xed\x95\x9c \xeb\xb2\x88\xec\x97\x90 \xeb\xa1\x9c\xeb\x93\x9c\xeb\x90\x98\xeb\x8a\x94 \xeb\x8d\xb0\xec\x9d\xb4\xed\x84\xb0 \xed\x81\xac\xea\xb8\xb0:', 128)
('label: ', array(0))
('text: ', (21,))
('label: ', array(0))
('text: ', (21,))
('label: ', array(0))
('text: ', (21,))
[3]
('\xed\x95\x9c \xeb\xb2\x88\xec\x97\x90 \xeb\xa1\x9c\xeb\x93\x9c\xeb\x90\x98\xeb\x8a\x94 \xeb\x8d\xb0\xec\x9d\xb4\xed\x84\xb0 \xed\x81\xac\xea\xb8\xb0:',

# 4. 모델 선언

In [38]:
#Recurrent neural network (many to one)
class RNN(nn.Module):
    def __init__(self, 
                input_size, 
                word_vec_size, 
                hidden_size, 
                n_classes, 
                num_layers = 4, 
                dropout_p = 0.3
               ):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.word_vec_size = word_vec_size
        self.hidden_size = hidden_size
        self.n_classes = n_classes
        self.num_layers = num_layers
        self.dropout_p = dropout_p
        
        self.emb = nn.Embedding(input_size, word_vec_size)
        self.lstm = nn.LSTM(input_size = word_vec_size,
                           hidden_size = hidden_size,
                            num_layers = num_layers,
                            dropout=dropout_p,
                            batch_first = True,
                            bidirectional = True)
        self.fc = nn.Linear(hidden_size*2, num_classes)
        
        self.activation = nn.LogSoftmax(dim=-1)
    def forward(self, x):
        x = self.emb(x)
        
        x, _ = self.lstm(x)
        
        out = self.activation(self.fc(x[:,-1]))
        
        return out

In [39]:
model = RNN(input_size = vocab_size, 
            word_vec_size = word_vec_size, 
            hidden_size = hidden_size, 
            n_classes = num_classes, 
            num_layers = num_layers, 
            dropout_p = dropout_p)

In [40]:
def ComputeAccr(dloader, imodel):
    correct = 0
    total = 0
    
    model.eval()
    for i, data in enumerate(dloader):
        texts = data.text.to(device)
        labels = data.label.to(device)
        
        output = model(texts)
        _, output_index = torch.max(output, 1)
        
        total += labels.size(0)
        correct += (output_index == labels).sum().float()
        
    model.train()
    return (100*correct/total).numpy()

In [41]:
print("Accuracy of Test Data: %.2f" %ComputeAccr(loaders.valid_loader, model))

Accuracy of Test Data: 89.03


# 5. loss, optimizer

In [42]:
loss_func = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [45]:
total_step = len(loaders.train_loader)
for epoch in range(num_epochs):
    for i, data in enumerate(loader.train_loader):
        texts = data.text.to(device)
        labels = data.label.to(device)
        
        print("[%d]" %i)
        
        outputs = model(texts)
        loss = loss_func(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accr: {:.2f}'
                 .format(epoch+1, num_epochs, i+1, total_step, loss.item(), ComputeAccr(loaders.valid_loader, model)))

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [1/10], Step [10/30], Loss: 0.4073, Accr: 89.03
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [1/10], Step [20/30], Loss: 0.0867, Accr: 89.03
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [1/10], Step [30/30], Loss: 0.2213, Accr: 89.03
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [2/10], Step [10/30], Loss: 0.4348, Accr: 78.39
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [2/10], Step [20/30], Loss: 2.2580, Accr: 93.23
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [2/10], Step [30/30], Loss: 0.2258, Accr: 89.03
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [3/10], Step [10/30], Loss: 0.1523, Accr: 89.35
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [3/10], Step [20/30], Loss: 0.1738, Accr: 92.80
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [3/10], Step [30/30], Loss: 0.0515, Accr: 95.05
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [4/10], Step [10/30], Loss: 0.4765, Accr: 93.98


In [46]:
print("Accuracy ouf Valid Data: %.2f" %ComputeAccr(loaders.valid_loader, model))

Accuracy ouf Valid Data: 97.10


In [49]:
netname = './rnn_weight.pkl'
torch.save(model, netname,)

/home/ec2-user/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
